In [76]:
import pandas as pd
import numpy as np
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["stepup"]
collection = db["shoes"]

# Fetch data from MongoDB
cursor = collection.find({})

# Create a DataFrame from MongoDB cursor
df = pd.DataFrame(list(cursor))

df.head()


,_id,shoeDetails,additionalFeatures,addedTimestamp
0,66901a5cf0933ae3d962947b,"{'size': '13', 'color': 'Black', 'brand': 'Adi...","{'hasLaces': True, 'isAthletic': True}",2024-07-11T20:53:12.519940
1,66901a5cf0933ae3d962947c,"{'size': '12', 'color': 'Brown', 'brand': 'H&M...","{'hasLaces': False, 'isAthletic': False}",2024-07-11T20:53:12.519940
2,66901a5cf0933ae3d962947d,"{'size': '10', 'color': 'Blue', 'brand': 'Vans...","{'hasLaces': True, 'isAthletic': False}",2024-07-11T20:53:12.519940


In [77]:
import pandas as pd   
import numpy as np 


#load dataset
df = pd.read_json('C:\Capstone_OnlineStore\python\shoes.json')

# Check the columns of your DataFrame
print(df)

     shoe_id                                        shoeDetails  \
0          0  {'brand': 'Vans', 'shoe_type': 'formal', 'colo...   
1          1  {'brand': 'Converse', 'shoe_type': 'athletic',...   
2          2  {'brand': 'Vans', 'shoe_type': 'athletic', 'co...   
3          3  {'brand': 'UGG', 'shoe_type': 'formal', 'color...   
4          4  {'brand': 'Adidas', 'shoe_type': 'sandals', 'c...   
..       ...                                                ...   
995      995  {'brand': 'Converse', 'shoe_type': 'athletic',...   
996      996  {'brand': 'Birkenstock', 'shoe_type': 'formal'...   
997      997  {'brand': 'Vans', 'shoe_type': 'boots', 'color...   
998      998  {'brand': 'Adidas', 'shoe_type': 'formal', 'co...   
999      999  {'brand': 'Crocs', 'shoe_type': 'formal', 'col...   

                                    additionalFeatures  
0    {'isPopular': True, 'inStock': True, 'onSale':...  
1    {'isPopular': False, 'inStock': False, 'onSale...  
2    {'isPopular': True,

<>:6: SyntaxWarning: invalid escape sequence '\C'
<>:6: SyntaxWarning: invalid escape sequence '\C'
C:\Users\wasadmin\AppData\Local\Temp\ipykernel_11412\4150060251.py:6: SyntaxWarning: invalid escape sequence '\C'
  df = pd.read_json('C:\Capstone_OnlineStore\python\shoes.json')


In [84]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

df = pd.read_json("shoes.json")

# pick relevant columns of choice
df_selected = df[['shoe_id', 'shoeDetails', 'additionalFeatures']].copy()

# Extract specific values from nested dictionaries
df_selected['brand'] = df_selected['shoeDetails'].apply(lambda x: x['brand'])
df_selected['shoe_type'] = df_selected['shoeDetails'].apply(lambda x: x['shoe_type'])
df_selected['color'] = df_selected['shoeDetails'].apply(lambda x: x['color'])
df_selected['size'] = df_selected['shoeDetails'].apply(lambda x: x['size'])
df_selected['price'] = df_selected['shoeDetails'].apply(lambda x: x['price'])
df_selected['isPopular'] = df_selected['additionalFeatures'].apply(lambda x: x['isPopular'])

# dropping redundant columns
df_selected.drop(['shoeDetails', 'additionalFeatures'], axis=1, inplace=True)

# categorical vars application of one-hot encoding 
df_encoded = pd.get_dummies(df_selected, columns=['brand', 'shoe_type', 'color', 'isPopular'])

knn = NearestNeighbors(n_neighbors=3).fit(df_encoded)

print(knn)

NearestNeighbors(n_neighbors=3)


In [85]:
selected_product_features = df_encoded[df_encoded['shoe_id'] == 1]
selected_product_index = selected_product_features.index[0]
distances, indices = knn.kneighbors(selected_product_features)
recommended_product_indices = indices.flatten()
recommended_products = df_selected.iloc[recommended_product_indices]

print("Selected Product:")
print(df_selected.iloc[selected_product_index])
print("\nRecommended Products:")
print(recommended_products)

Selected Product:
shoe_id             1
brand        Converse
shoe_type    athletic
color           white
size                7
price           199.3
isPopular       False
Name: 1, dtype: object

Recommended Products:
   shoe_id     brand shoe_type  color  size   price  isPopular
1        1  Converse  athletic  white     7  199.30      False
5        5     Crocs    casual  beige     9  194.85       True
2        2      Vans  athletic  beige     9  190.17       True
